In [1]:
from datetime import datetime
!pip install boto3
!pip install model-registry
import os
import boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 271.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 258.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 336.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 115.7 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 361.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 451.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 348.0 MB/s eta 0:

In [2]:
registeredmodel_name = "mnist"
version_name = "v1.nb"+datetime.now().strftime("%Y%m%d%H%M%S")
print(f"Will be using: {registeredmodel_name}:{version_name} in the remainder of this task")

Will be using: mnist:v1.nb20240325100512 in the remainder of this task


In [3]:
s3 = boto3.resource(
    service_name='s3',
    region_name=os.environ['AWS_DEFAULT_REGION'],
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    use_ssl=False,
    endpoint_url=os.environ['AWS_S3_ENDPOINT'],
    verify=False
)

In [4]:
bucket_name = os.environ['AWS_S3_BUCKET']
odh_secret_name = f'aws-connection-{bucket_name}'
in_bucket_path = version_name


In [5]:
in_bucket_target = f'{in_bucket_path}/mnist.onnx'
full_bucket_target = f's3://{bucket_name}/registry/{in_bucket_target}'
print({full_bucket_target})

{'s3://ods-ci-ds-pipelines/registry/v1.nb20240325100512/mnist.onnx'}


In [7]:
my_bucket = s3.Bucket(bucket_name)
my_bucket.upload_file("mnist.onnx", in_bucket_target)

print("ls:")
for obj in my_bucket.objects.filter():
    print(obj.key)

ls:
08bb1477-9b56-432a-b97a-b046d5e9f555
0dec4b73-dc89-4540-af70-0142f9b62b4b
111b4583-2f83-4977-b4ff-6055a429fe3b
1a5fd366-894a-462b-9ca8-d7c37d92a9c4
1a8be383-9900-411b-9106-879e316b3817
1d2d0946-fd72-411b-b5ac-47c20aaaba17
204c53d3-dca6-4266-8256-274404bc4b57
20d6f616-31c1-4abb-9bf6-000d530c67c6
24607d5f-67d9-477a-8426-0d7efb20bcd7
247ab2ff-57c5-4d78-a266-3ee4959f98d2
27bfa778-6129-46dd-a0e9-fdde4f04a73d
3bf9d41f-6ac4-4a8d-bb39-658fb660cc80
3c5bbafa-25fd-4e08-b1e2-188345bff247
4bf64650-fe33-463c-9664-92e61bfdafed
5946ae81-96a2-4ddf-8443-864eb63bdcbb
5b1f1b4b-a8b1-49fe-ba52-00852c9f5def
5b6473e2-e54a-4177-b9b5-2aa69fff436f
5caa6ad4-265f-401a-9e68-de7333912971
6 Train Save-0301104150/1_experiment_train-fd0d8464-1fa6-4d29-9da5-2b185a7cc8f3.tar.gz
6 Train Save-0301104150/1_experiment_train.html
6 Train Save-0301104150/1_experiment_train.ipynb
6 Train Save-0301104150/2_save_model-4fd1c6cc-93e1-4294-ae0a-2d1125cff8ca.tar.gz
6 Train Save-0301104150/2_save_model.html
6 Train Save-0301104150

In [8]:
files = os.listdir(os.curdir) 
print(files)


['.cache', 'mnist.onnx', 'lost+found', '.ipynb_checkpoints', '.jupyter', '.ipython', 'Untitled.ipynb', '.local']


In [18]:
from model_registry import ModelRegistry

registry = ModelRegistry(server_address="modelregistry-sample.model-registry-project.svc.cluster.local", port=9090, author="author")

In [17]:
from model_registry import ModelRegistry

registry = ModelRegistry(server_address="model-registry-project", port=9090, author="author")

In [19]:
rm = registry.register_model(registeredmodel_name,
                                full_bucket_target,
                                model_format_name="onnx",
                                model_format_version="1",
                                storage_key=odh_secret_name,
                                storage_path=in_bucket_path,
                                version=version_name,
                                description="demo20231121 e2e MNIST",
                                )

In [20]:
print("RegisteredModel:")
print(registry.get_registered_model(registeredmodel_name))
print("ModelVersion:")
print(registry.get_model_version(registeredmodel_name, version_name))
print("ModelArtifact:")
print(registry.get_model_artifact(registeredmodel_name, version_name))

RegisteredModel:
RegisteredModel(id='1', description='', external_id='', create_time_since_epoch=1711374638377, last_update_time_since_epoch=1711374638377, state=<ContextState.LIVE: 'LIVE'>, name='mnist')
ModelVersion:
ModelVersion(name='v1.nb20240325100512', id='2', description='demo20231121 e2e MNIST', external_id='', create_time_since_epoch=1711374638978, last_update_time_since_epoch=1711374638978, state=<ContextState.LIVE: 'LIVE'>, model_name='mnist', version='v1.nb20240325100512', author='author', metadata={'AWS_S3_ENDPOINT': 'https://s3.amazonaws.com/', 'AWS_S3_BUCKET': 'ods-ci-ds-pipelines', 'AWS_DEFAULT_REGION': 'us-east-1'}, artifacts=NOTHING, _registered_model_id=NOTHING)
ModelArtifact:
ModelArtifact(id='1', description='', external_id='', create_time_since_epoch=1711374639835, last_update_time_since_epoch=1711374639835, name='mnist', uri='s3://ods-ci-ds-pipelines/registry/v1.nb20240325100512/mnist.onnx', state=<ArtifactState.UNKNOWN: 0>, model_format_name='onnx', model_forma